# 0.reRank_reOrder_main 비교 

##   Ollama-ko-0710:latest 와 gpt-4.1-mini 비교

### ollama-ko-0710:latest 답변

- 질문: 삼성전자 하반기 실적 전망은?

- 답변:'삼성전자의 하반기 실적 전망은 긍정적인 방향을 보입니다. 최근 몇 주간의 주가 상승과 시장 기대에 기반한 것으로, 특히 반도체 부문에서 나온 성과가 크게 기여하고 있습니다. KB증권 리서치본부장 김동원 박사의 분석에서도 삼성전자는 밸류에이션 측면에서 SK하이닉스 대비 여전히 디스카운트 상태이며, 여러 조건이 맞아떨어지면 추가 상승 여력이 있다고 언급되었습니다.

또한, 최근의 주가 움직임을 보면 9월 초반에도 삼성전자는 계속해서 상승세를 이어갔으며, 특히 외국인과 기관 투자자의 매수가 활발하게 이루어지고 있습니다. 또한, 코스피 시가총액 상위 종목으로서의 성장도 긍정적인 요소입니다.

그러나, 반도체 시장은 불확실성이 높은 분야이므로, 업황 변화에 따라 실적은 변동될 수 있습니다. 그러나 현재까지의 전망과 최근 주가 움직임을 보면 삼성전자의 하반기 실적은 호실적으로 전망됩니다. 추가적인 분석이나 보고서를 통해 더 구체적인 전망을 얻을 수 있을 것입니다.

### gpt-4.1-mini 

- 질문: 삼성전자 하반기 실적 전망은?

- 답변: '삼성전자의 하반기 실적 전망은 긍정적입니다. 3분기 영업이익은 약 9조 원으로, 1년 전보다는 1.9% 감소하나 직전 분기 대비로는 91.9% 증가할 것으로 예상됩니다. 또한, 미래에셋증권은 올해와 내년 삼성전자의 영업이익 전망치를 각각 2.9%, 5.3% 상향 조정하였고, 목표 주가도 8만8000원에서 9만6000원으로 9% 올렸습니다. 이는 내년 메모리 반도체 공급 부족과 고대역폭 메모리(HBM) 생산 집중 덕분에 D램 가격 상승이 기대되기 때문입니다. 따라서 삼성전자는 중장기적으로도 경쟁력 회복과 밸류에이션 정상화가 기대되는 상황입니다'

### 결과

- 질문 : 삼성전자 하반기 실적 전망은?
- 데이터 : 데이터 모두 삼성전자 상반기 정보 없음

### retriver 생성
```
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
)
```

### ReRanker
```
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=reranker, top_n=6)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
```
### reOrder
```
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs
```

### chain
```
  chain = (
    {
        "context": itemgetter("question")
        | compression_retriever
        | RunnableLambda(reorder_documents),  # 질문을 기반으로 문맥을 검색합니다.
        "question": itemgetter("question"),  # 질문을 추출합니다.
    }
    | prompt  # 프롬프트 템플릿에 값을 전달합니다.
    | llm # 언어 모델에 프롬프트를 전달합니다.
    | StrOutputParser()  # 모델의 출력을 문자열로 파싱합니다.
    )

```

- context : 같은 데이터 증강
- 답변 품질 : 두 답변 모두 적절함
- perplexity 평가 : gpt 모델의 답변이 더 구체적이며, 설득력이 높다고 판단함

- 총평 : 성능 뿐아니라 속도 측면에서도 gpt 모델이 매우 우수함
